### 주제: CarDecko 사이트에 올라온 중고차 가격을 회귀 예측
- CarDecko는 인도의 중고차 관련 웹 사이트

---

#### Features (총 65개)


#### Targets
- listed_price: 판매 사이트에 제시된 가격

In [15]:
import pandas as pd
import numpy as np

# 데이터 세트 불러오기
c_df = pd.read_csv('../datasets/cars_data_clean.csv', low_memory=False)
c_df

,usedCarSkuId,loc,myear,body,transmission,fuel,km,ip,images,imgCount,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
0,7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,cng,69162.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,NaN,NaN,NaN,NaN,58.16,6200.0,77.0,3500.0,69.0,NaN
1,c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,cng,45864.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,Gasoline Port Injection,11.0,NaN,NaN,58.20,6000.0,78.0,3500.0,73.0,NaN
2,7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,cng,81506.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,NaN,NaN,NaN,NaN,86.70,6000.0,109.0,4500.0,NaN,NaN
3,278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,cng,115893.0,0,[{'img': ''}],0,...,Multi-Point Fuel Injection,NaN,13.0,NaN,58.20,6200.0,77.0,3500.0,NaN,NaN
4,b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,cng,18900.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,...,NaN,12.0,NaN,NaN,86.63,5500.0,121.5,4200.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37808,ee6ce523-2aa6-4852-96a1-86b9bc277377,thathawade,2018,suv,manual,diesel,49882.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,Common Rail Injection,NaN,16.0,209.0,108.50,3750.0,260.0,2125.0,76.0,82.5
37809,ba1994ac-53ab-4baf-913e-b72f01a37b5d,kharadi,2011,sedan,manual,diesel,82843.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,27,...,Common Rail Injection,NaN,15.0,NaN,88.80,4000.0,200.0,2375.0,NaN,NaN
37810,8f4448ae-53f5-4a3f-b13d-2401dd60c624,kothrud,2020,hatchback,manual,diesel,58000.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,11,...,NaN,NaN,16.0,NaN,88.77,4000.0,200.0,2125.0,NaN,NaN
37811,dbf9b629-7215-494b-a62d-ba1408531362,kothrud,2014,hatchback,manual,diesel,62000.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,8,...,Common Rail Injection,NaN,NaN,NaN,69.00,4000.0,140.0,2400.0,NaN,NaN


In [16]:
c_df.discountValue.unique()

array([     0,  24000,   6000,   3000,  30000,  94000,   5000,  65000,
       120000,   7000,  10000, 100000,  36000,   4000,  11000, 200000,
       160000,  57000,  37000,  32000,  21000,  50000,  60000, 130000,
       140000,  14000, 165000,  55000,      1,  15000], dtype=int64)

In [17]:
c_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37813 entries, 0 to 37812
Data columns (total 66 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   usedCarSkuId              37813 non-null  object 
 1   loc                       31963 non-null  object 
 2   myear                     37813 non-null  int64  
 3   body                      37794 non-null  object 
 4   transmission              37813 non-null  object 
 5   fuel                      37813 non-null  object 
 6   km                        37813 non-null  float64
 7   ip                        37813 non-null  int64  
 8   images                    37813 non-null  object 
 9   imgCount                  37813 non-null  int64  
 10  threesixty                37813 non-null  bool   
 11  dvn                       37813 non-null  object 
 12  oem                       37813 non-null  object 
 13  model                     37813 non-null  object 
 14  varian

#### 현재 데이터 세트에 필요한 것들
- 불필요한 컬럼 제거
- 결측치 조정: 데이터가 고작 513개인 컬럼이 있기 때문에 제거는 생각해볼 것. 참고로 중복 데이터는 없음
- 차원 축소: **프로젝트의 목적**. 자세한 방식은 로지스틱 회귀 프로젝트 참고할 것
- 인코딩: 문자열 컬럼은 unique()로 클래스 종류 확인해서 인코딩 방식 정할 것
- 표준화 후 이상치 제거: 꿀 가격 예측 프로젝트 참고. 원래 수치형인 컬럼들만 따로 뺄 것

### 데이터 전처리

In [18]:
# 데이터 세트 복사
pre_c_df = c_df.copy()
pre_c_df

,usedCarSkuId,loc,myear,body,transmission,fuel,km,ip,images,imgCount,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
0,7111bf25-97af-47f9-867b-40879190d800,gomti nagar,2016,hatchback,manual,cng,69162.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,NaN,NaN,NaN,NaN,58.16,6200.0,77.0,3500.0,69.0,NaN
1,c309efc1-efaf-4f82-81ad-dcb38eb36665,borivali west,2015,hatchback,manual,cng,45864.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,Gasoline Port Injection,11.0,NaN,NaN,58.20,6000.0,78.0,3500.0,73.0,NaN
2,7609f710-0c97-4f00-9a47-9b9284b62d3a,jasola,2015,sedan,manual,cng,81506.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,NaN,NaN,NaN,NaN,86.70,6000.0,109.0,4500.0,NaN,NaN
3,278b76e3-5539-4a5e-ae3e-353a2e3b6d7d,jasola,2013,hatchback,manual,cng,115893.0,0,[{'img': ''}],0,...,Multi-Point Fuel Injection,NaN,13.0,NaN,58.20,6200.0,77.0,3500.0,NaN,NaN
4,b1eab99b-a606-48dd-a75b-57feb8a9ad92,mumbai g.p.o.,2022,muv,manual,cng,18900.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,6,...,NaN,12.0,NaN,NaN,86.63,5500.0,121.5,4200.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37808,ee6ce523-2aa6-4852-96a1-86b9bc277377,thathawade,2018,suv,manual,diesel,49882.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,15,...,Common Rail Injection,NaN,16.0,209.0,108.50,3750.0,260.0,2125.0,76.0,82.5
37809,ba1994ac-53ab-4baf-913e-b72f01a37b5d,kharadi,2011,sedan,manual,diesel,82843.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,27,...,Common Rail Injection,NaN,15.0,NaN,88.80,4000.0,200.0,2375.0,NaN,NaN
37810,8f4448ae-53f5-4a3f-b13d-2401dd60c624,kothrud,2020,hatchback,manual,diesel,58000.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,11,...,NaN,NaN,16.0,NaN,88.77,4000.0,200.0,2125.0,NaN,NaN
37811,dbf9b629-7215-494b-a62d-ba1408531362,kothrud,2014,hatchback,manual,diesel,62000.0,0,[{'img': 'https://images10.gaadi.com/usedcar_i...,8,...,Common Rail Injection,NaN,NaN,NaN,69.00,4000.0,140.0,2400.0,NaN,NaN


In [19]:
pre_c_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37813 entries, 0 to 37812
Data columns (total 66 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   usedCarSkuId              37813 non-null  object 
 1   loc                       31963 non-null  object 
 2   myear                     37813 non-null  int64  
 3   body                      37794 non-null  object 
 4   transmission              37813 non-null  object 
 5   fuel                      37813 non-null  object 
 6   km                        37813 non-null  float64
 7   ip                        37813 non-null  int64  
 8   images                    37813 non-null  object 
 9   imgCount                  37813 non-null  int64  
 10  threesixty                37813 non-null  bool   
 11  dvn                       37813 non-null  object 
 12  oem                       37813 non-null  object 
 13  model                     37813 non-null  object 
 14  varian

In [45]:
pre_c_df[['Ground Clearance Unladen']]

,Ground Clearance Unladen
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
37808,209.0
37809,NaN
37810,NaN
37811,NaN


In [41]:
pre_c_df['model_type_new'].unique()

array(['used'], dtype=object)

In [47]:
# 불필요한 컬럼들 삭제
columns = ['usedCarSkuId', 'loc', 'images', 'imgCount', 'threesixty', 'dvn', 'City',
           'top_features', 'comfort_features', 'interior_features', 'exterior_features',
           'safety_features', 'Color', 'Valve Configuration', 'model_type_new', 'state', 'exterior_color']

pre_c_df = pre_c_df.drop(labels=columns, axis=1)
pre_c_df

,myear,body,transmission,fuel,km,ip,oem,model,variant,listed_price,...,Fuel Suppy System,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke
0,2016,hatchback,manual,cng,69162.0,0,maruti,maruti wagon r,lxi cng,370000.0,...,NaN,NaN,NaN,NaN,58.16,6200.0,77.0,3500.0,69.0,NaN
1,2015,hatchback,manual,cng,45864.0,0,maruti,maruti celerio,green vxi,365000.0,...,Gasoline Port Injection,11.0,NaN,NaN,58.20,6000.0,78.0,3500.0,73.0,NaN
2,2015,sedan,manual,cng,81506.0,0,honda,honda amaze,s plus i-vtec,421000.0,...,NaN,NaN,NaN,NaN,86.70,6000.0,109.0,4500.0,NaN,NaN
3,2013,hatchback,manual,cng,115893.0,0,maruti,maruti wagon r,lxi cng,240000.0,...,Multi-Point Fuel Injection,NaN,13.0,NaN,58.20,6200.0,77.0,3500.0,NaN,NaN
4,2022,muv,manual,cng,18900.0,0,maruti,maruti ertiga,vxi cng,1175000.0,...,NaN,12.0,NaN,NaN,86.63,5500.0,121.5,4200.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37808,2018,suv,manual,diesel,49882.0,0,tata,tata nexon,1.5 revotorq xz plus,881000.0,...,Common Rail Injection,NaN,16.0,209.0,108.50,3750.0,260.0,2125.0,76.0,82.5
37809,2011,sedan,manual,diesel,82843.0,0,tata,tata manza,aura quadrajet bs iv,245000.0,...,Common Rail Injection,NaN,15.0,NaN,88.80,4000.0,200.0,2375.0,NaN,NaN
37810,2020,hatchback,manual,diesel,58000.0,0,tata,tata altroz,xz diesel,735000.0,...,NaN,NaN,16.0,NaN,88.77,4000.0,200.0,2125.0,NaN,NaN
37811,2014,hatchback,manual,diesel,62000.0,0,tata,tata indica ev2,ls,225000.0,...,Common Rail Injection,NaN,NaN,NaN,69.00,4000.0,140.0,2400.0,NaN,NaN


In [48]:
# 타겟 데이터를 맨 끝으로 이동
pre_c_df.loc[:, 'target'] = pre_c_df.listed_price
pre_c_df = pre_c_df.drop(labels=['listed_price'], axis=1)
pre_c_df

,myear,body,transmission,fuel,km,ip,oem,model,variant,discountValue,...,Compression Ratio,Alloy Wheel Size,Ground Clearance Unladen,Max Power Delivered,Max Power At,Max Torque Delivered,Max Torque At,Bore,Stroke,target
0,2016,hatchback,manual,cng,69162.0,0,maruti,maruti wagon r,lxi cng,0,...,NaN,NaN,NaN,58.16,6200.0,77.0,3500.0,69.0,NaN,370000.0
1,2015,hatchback,manual,cng,45864.0,0,maruti,maruti celerio,green vxi,0,...,11.0,NaN,NaN,58.20,6000.0,78.0,3500.0,73.0,NaN,365000.0
2,2015,sedan,manual,cng,81506.0,0,honda,honda amaze,s plus i-vtec,0,...,NaN,NaN,NaN,86.70,6000.0,109.0,4500.0,NaN,NaN,421000.0
3,2013,hatchback,manual,cng,115893.0,0,maruti,maruti wagon r,lxi cng,0,...,NaN,13.0,NaN,58.20,6200.0,77.0,3500.0,NaN,NaN,240000.0
4,2022,muv,manual,cng,18900.0,0,maruti,maruti ertiga,vxi cng,0,...,12.0,NaN,NaN,86.63,5500.0,121.5,4200.0,NaN,NaN,1175000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37808,2018,suv,manual,diesel,49882.0,0,tata,tata nexon,1.5 revotorq xz plus,0,...,NaN,16.0,209.0,108.50,3750.0,260.0,2125.0,76.0,82.5,881000.0
37809,2011,sedan,manual,diesel,82843.0,0,tata,tata manza,aura quadrajet bs iv,0,...,NaN,15.0,NaN,88.80,4000.0,200.0,2375.0,NaN,NaN,245000.0
37810,2020,hatchback,manual,diesel,58000.0,0,tata,tata altroz,xz diesel,0,...,NaN,16.0,NaN,88.77,4000.0,200.0,2125.0,NaN,NaN,735000.0
37811,2014,hatchback,manual,diesel,62000.0,0,tata,tata indica ev2,ls,0,...,NaN,NaN,NaN,69.00,4000.0,140.0,2400.0,NaN,NaN,225000.0


#### 중간 분석
- 현재 타겟은 연속형
- 따라서 차원 축소 방식은 무조건 PCA
> - LDA는 클래스 별로 나누는 것이 중점이라, 타겟 데이터가 연속형인 이번 프로젝트에서는 사용 불가

#### 결측치 조정
- 현재 결측치 수가 절대 다수인 컬럼들 존재
- describe()로 수치 데이터 확인 후 전처리 바람

In [49]:
# 결측치 확인
pre_c_df.isna().sum()

myear                           0
body                           19
transmission                    0
fuel                            0
km                              0
ip                              0
oem                             0
model                           0
variant                         0
discountValue                   0
utype                           0
carType                         0
Engine Type                   868
No of Cylinder                143
Valves per Cylinder           228
Turbo Charger                   0
Super Charger                   0
Length                        362
Width                         371
Height                        363
Wheel Base                    556
Front Tread                 15874
Rear Tread                  15875
Kerb Weight                 13139
Gross Weight                20890
Gear Box                      471
Drive Type                   4497
Seats                          18
Steering Type                 808
Turning Radius